<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/AI_Forced_Alignment_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# intel-tensorflow is required to run on CPU
!pip install torch torchaudio soundfile omegaconf intel-tensorflow tensorflow_hub

In [55]:
#speech_file = 'speech_orig.wav'
speech_file = '/content/recorded.wav'


import os
import torch
import tensorflow as tf
import tensorflow_hub as tf_hub
from omegaconf import OmegaConf

language = 'en'

# load provided utils using torch.hub for brevity
_, decoder, utils = torch.hub.load('snakers4/silero-models', model='silero_stt', language=language)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils

# see available models
torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml', 'models.yml')
models = OmegaConf.load('models.yml')
available_languages = list(models.stt_models.keys())
assert language in available_languages

# load the actual tf model
tf_model = tf_hub.load(models.stt_models.get(language).latest.tf)

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file(models.stt_models.get(language).latest.meta.sample, dst = speech_file, progress=True)
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master



the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [56]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# tf inference
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))

the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [57]:
batches

[['/content/recorded.wav']]

In [58]:
tf_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fbe7c5c09b0>

In [59]:
res

<tf.Tensor: shape=(1, 136, 999), dtype=float32, numpy=
array([[[9.9562067e-01, 2.1983162e-06, 4.2956495e-03, ...,
         7.4658083e-06, 6.9315170e-06, 1.2493901e-05],
        [2.3445139e-02, 1.3809478e-05, 9.7602081e-01, ...,
         3.1058391e-06, 1.8454934e-06, 3.6289461e-05],
        [2.6382526e-02, 6.0946086e-08, 8.4972737e-05, ...,
         1.2406174e-05, 1.3228331e-06, 9.7339547e-01],
        ...,
        [9.9961782e-01, 1.4285623e-06, 9.0514386e-06, ...,
         6.1206483e-05, 6.5136401e-06, 2.0248130e-05],
        [9.9961782e-01, 1.4285079e-06, 9.0514040e-06, ...,
         6.1204380e-05, 6.5133045e-06, 2.0246334e-05],
        [9.9961782e-01, 1.4285732e-06, 9.0529056e-06, ...,
         6.1196843e-05, 6.5128511e-06, 2.0243941e-05]]], dtype=float32)>

In [60]:
decoder

In [61]:
# construct a shape[1,136,999] tensor where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 0
the_tensor = []
for i in range(136):
  if i == value:
    the_tensor.append(list(np.ones(999)))
  else:
    the_tensor.append(list(np.zeros(999)))

the_tensor = np.array(the_tensor)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)


tf.Tensor(
[[[1. 1. 1. ... 1. 1. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)


In [62]:
#res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(the_tensor.numpy())[0]))

# Test that the 999 does refer to time, and thus halving it will shorten the output
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
print(decoder(torch.Tensor(res.numpy())[0]))


the birch canoe slit on the smooth planks blared the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [63]:
res1, res2 = tf.split(res, [500,499],2)
print(res1.shape)
#print(decoder(torch.Tensor(res1.numpy())[0]))
'''---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<ipython-input-38-617167afc803> in <module>()
      1 res1, res2 = tf.split(res, [500,499],2)
      2 print(res1.shape)
----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))

1 frames
/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)
     62     def process(self,
     63                 probs, wav_len, word_align):
---> 64         assert len(self.labels) == probs.shape[1]
     65         for_string = []
     66         argm = torch.argmax(probs, axis=1)

AssertionError: '''

(1, 136, 500)


'---------------------------------------------------------------------------\nAssertionError                            Traceback (most recent call last)\n<ipython-input-38-617167afc803> in <module>()\n      1 res1, res2 = tf.split(res, [500,499],2)\n      2 print(res1.shape)\n----> 3 print(decoder(torch.Tensor(res1.numpy())[0]))\n\n1 frames\n/root/.cache/torch/hub/snakers4_silero-models_master/utils.py in process(self, probs, wav_len, word_align)\n     62     def process(self,\n     63                 probs, wav_len, word_align):\n---> 64         assert len(self.labels) == probs.shape[1]\n     65         for_string = []\n     66         argm = torch.argmax(probs, axis=1)\n\nAssertionError: '

In [64]:
# construct a shape[1,136,999] tensore where the columns consist of a single 1 in one of the 136 and the others 0
# In this way I may find out what the 136 means

import numpy as np
value = 11
the_tensor = []
for i in range(999):
  if i == value:
    temp = list(np.ones(136))
    temp[10] = 0
    the_tensor.append(temp)
  else:
    temp = list(np.zeros(136))
    the_tensor.append(temp)
    if i == 12:
      temp[10] = 1


the_tensor = np.array(the_tensor)
the_tensor = np.swapaxes(the_tensor, 0, 1)
the_tensor = the_tensor[np.newaxis, :]
#print(the_tensor.shape)
the_tensor = tf.convert_to_tensor(the_tensor)
print(the_tensor)

print(decoder(torch.Tensor(the_tensor.numpy())[0]))

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 136, 999), dtype=float64)
entoen


In [65]:
#@title Install dependencies

!pip install -q omegaconf
!pip install -q torchaudio
!pip install -q soundfile
!pip install -q pydub

import os
from os.path import exists

if not exists('silero-models'):
  !git clone -q --depth 1 https://github.com/snakers4/silero-models

%cd silero-models

# silero imports
import torch
import random
from glob import glob
from omegaconf import OmegaConf
from utils import (init_jit_model, 
                   split_into_batches,
                   read_audio,
                   read_batch,
                   prepare_model_input)
from colab_utils import (record_audio,
                         audio_bytes_to_np,
                         upload_audio)

device = torch.device('cpu')   # you can use any pytorch device
models = OmegaConf.load('models.yml')

# imports for uploading/recording
import numpy as np
import ipywidgets as widgets
from scipy.io import wavfile
from IPython.display import Audio, display, clear_output
from torchaudio.functional import vad


/content/silero-models/silero-models


In [66]:
sample_rate = 16000

clear_output()
audio = record_audio(10)
wavfile.write('/content/recorded.wav', sample_rate, (32767*audio).numpy().astype(np.int16))


#button = widgets.Button(description="Record Speech")
#button.on_click(_record_audio)
#display(button)

display(Audio(audio, rate=sample_rate, autoplay=True))
wavfile.write('test.wav', sample_rate, (32767*audio).numpy().astype(np.int16))
  

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [67]:
len(decoder.labels)

999

In [68]:
decoder.blank_idx

0

In [69]:
speech_file = '/content/recorded.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'___________________________________________________________________________________no_ise___________________________________'

In [70]:
result = np.argmax(res_array, axis=2)

#result = f(result)

In [71]:
result[0].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 33,
 0,
 408,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [72]:
#Testing singing
speech_file = '/content/vocals.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)

'___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________what_  _not   _believe__       __eyes_____  if   can_ _million  _f__ire_   f_li________ _when       the all_   as i  fellll____ly_____________________________________because  _they  _fell      open____ ________    le_    your ddro_ps _every____where_   you   _  me_   with   but   i  would  _just  stand_  and________  _st_____________________________i_  _like    make_   my____self_______________________________________________________________________________________________________ _it  _hard_ to say_     i  _ratherther__________________  i_ ___________________ _because_  _every_______  is   _never_______  to _see_________________________________________i   _get   a _thousand___  ____________ from   and  thousand__ 

In [73]:
decoder.blank_idx

0

In [74]:
np.argsort(res_array, axis=2)[0][100]
#result = np.argmax(res_array, axis=2)
#result[0][100]

array([685, 822, 617, 653, 765, 902, 442, 803, 833, 821, 769, 876, 836,
       750, 867, 749, 306, 493, 635, 699, 797, 600, 913, 572, 908, 649,
       598, 580, 962, 691, 928, 942, 623, 660, 948, 954, 824, 530, 915,
       841, 877, 963, 891, 714, 707, 596, 504, 485, 589, 786, 353, 853,
       633, 730, 243, 723, 857, 820, 944, 647, 818, 905, 741, 943, 745,
       914, 811, 856, 620, 618, 420, 658, 209, 804, 502, 782, 586, 298,
       879, 687, 753, 585, 295, 958, 862, 768, 783, 631, 892, 775, 362,
       855, 900, 743, 935, 810, 684, 924, 778, 940, 854, 434, 497, 701,
       590, 827, 830, 937, 358, 903, 708, 603, 568, 837, 646, 956, 611,
       540, 483, 930, 424, 711, 784, 878, 910, 965, 654, 716, 889, 636,
       843, 737, 847, 792, 508, 834, 712, 852, 875, 479, 297, 850, 445,
       933, 464, 957, 655, 901, 318, 670, 955, 380, 817, 870, 917, 605,
       275, 630, 622, 771, 871, 709, 562, 796, 652, 613, 593, 602, 637,
       851, 323, 441, 601, 726, 805, 528, 381, 746, 512, 429, 84

In [75]:



#Testing singing with empty removed
speech_file = '/content/fireflies_voc_1min.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

original = result[0].copy()
for i,val in enumerate(original):
  #print(val)
  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


OSError: ignored

In [ ]:



#Testing singing with empty removed
speech_file = '/content/jepsen_voc.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

# For each frame print the grapheme that is the most likely
graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
res_array = np.array(res)
res_array[0].shape
result = np.argmax(res_array, axis=2)

#original = result[0].copy()
#for i,val in enumerate(original):
  #print(val)
#  if val == 0:
    #print(np.argsort(res_array, axis=2)[0][i][-2])
    #result[0][i] = np.argsort(res_array, axis=2)[0][i][-2]
#    result[0][i] = np.argpartition(res_array, -2, axis=2)[0][i][-2]

text = []
for i in result[0].tolist():
  text.append(graphemes[i])

"".join(text)


In [ ]:
decoder.labels

In [76]:
import math

subproblems = []
def minimal_token(text, tokens):
  global subproblems
  subproblems = [None for i in range(len(text))]
  # Solve all of the subproblems
  
  for i in range(len(text)):
    minimal_token_recursive(text, len(text) - 1 - i, tokens)
  solution = minimal_token_recursive(text, 0, tokens)
  return solution

def minimal_token_recursive(text, index, tokens):
  global subproblems
  if index >= len(text):
    return (0,[])
  # If we haven't calculated the solution before, calculate it now
  if subproblems[index] is None:
    subproblems[index] = minimal_token_recursive_calculate(text, index, tokens)
    
  return subproblems[index]
  


def minimal_token_recursive_calculate(text, index, tokens):
  minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = minimal_token_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)

  


In [ ]:
#minimal_token("the inovation sensation", decoder.labels)
minimal_token("the", decoder.labels)

In [ ]:
decoder.

In [ ]:
str.find("hello", "ll")

In [ ]:
"hello"[0:]

In [ ]:
k = [1,2]
k.insert(2,0)
k

In [ ]:
minimal_token("hello", ["he", "llo", "l", "lo", "o"])

In [ ]:
minimal_token("hello", ["he", "llo", "l", "lo"])

In [ ]:
test = [1].append(k)
test

In [ ]:
res.shape

In [ ]:
the_string = '''look at this stuff isnt it neat wouldn't you think my collection's complete wouldn't you think i'm the girl girl who has everything'''

minimal_token(the_string, decoder.labels)

In [115]:
import math


subproblems = []
space_index = decoder.labels.index(' ')
def forced_alignment(correct_text, text_from_speech):
  global subproblems
  subproblems = []

  time_values = np.array(text_from_speech).shape[0]

  for i in range(len(correct_text)):
    subproblems.append([])
    for j in range(time_values):
      subproblems[-1].append(None)

  for i in range(len(correct_text)):
    for j in range(time_values):
      forced_alignment_recursive(correct_text, text_from_speech, i, j)

  
  #solution = forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)

  solution = []
  value = len(correct_text) - 1
  for i in range(time_values):
    
    solution.append(value)
    time = time_values - 1 - i
    value -= subproblems[value][time][1]
    


  solution.reverse()

  return solution

def forced_alignment_recursive(correct_text, text_from_speech, text_index, time):
  global subproblems
  
  
  if time < text_index or text_index < 0:
    return (-math.inf, 0)
  
  

  # If we haven't calculated the solution before, calculate it now
  try:
    subproblems[text_index][time]
  except:
    print(text_index, time)

  if subproblems[text_index][time] is None:
    subproblems[text_index][time] = forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time)
    
  return subproblems[text_index][time]
  


def forced_alignment_recursive_calculate(correct_text, text_from_speech, text_index, time):
  if time == 0:
    return (0, 0)
  text = correct_text[text_index]
  # Replace spaces with the unknown symbol
  global space_index
  if text == space_index:
    text = 0

  probability = math.log(text_from_speech[time][text])

  


  

  
  options = [forced_alignment_recursive(correct_text, text_from_speech, text_index, time - 1),
             forced_alignment_recursive(correct_text, text_from_speech, text_index - 1, time - 1),
  ]

  option_probabilities = [i[0] for i in options]
  options_index = np.argmax(option_probabilities)

  #indexes = options[options_index][1].copy()
  #indexes.append(text_index)
  return (options[options_index][0] + probability, options_index)







  '''minimum = None
  minimum_token = None
  for i, token in enumerate(tokens):
    if str.find(text[index:], token) == 0:
      subproblem = forced_alignment_recursive(text, index+len(token), tokens)
      if subproblem is None:
        raise Exception("Sub none")
      if minimum is None or subproblem[0] < minimum[0]:
        minimum = subproblem
        minimum_token = i
  if minimum == None:
    #raise Exception("This text cannot be represented")
    # or instead just return something with a large cost
    return (math.inf, [])
  
  new_list = minimum[1].copy()
  new_list.insert(0, minimum_token)
  return (1 + minimum[0], new_list)'''

  


In [ ]:
test = [1,3]
np.argmax(test)

In [ ]:
math.log(5)

In [118]:
import string
text = '''you would not believe your eyes if ten million fireflies lit up the world as i fell asleep cause they fill the open air and leave teardrops everywhere you'd think me rude but i would just stand and stare i'd like to make myself believe that planet earth turns slowly it's hard to say that i'd rather stay awake when i'm asleep cause everything is never as it seems cause i'd get a thousand hugs from ten thousand lightning bugs as they tried to teach me how to dance a foxtrot above my head'''

text = '''You would not believe your eyes
If ten million fireflies
Lit up the world as I fell asleep
'Cause they fill the open air
And leave teardrops everywhere
You'd think me rude but I would just stand and stare
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems
'Cause I'd get a thousand hugs
From ten thousand lightning bugs
As they tried to teach me how to dance
A foxtrot above my head
A sock-hop beneath my bed
A disco ball is just hanging by a thread (thread, thread)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)
Leave my door open just a crack
Please take me away from here
'Cause I feel like such an insomniac
Please take me away from here
Why do I tire of counting sheep?
Please take me away from here
When I'm far too tired to fall asleep
To ten million fireflies
I'm weird 'cause I hate goodbyes
I got misty eyes as they said farewell (they said farewell)
But I'll know where several are
If my dreams get real bizarre
'Cause I saved a few and I keep them in a jar (jar, jar, jar)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)
I'd like to make myself believe that planet earth turns slowly
It's hard to say that I'd rather stay awake when I'm asleep
'Cause everything is never as it seems (when I fall asleep)'''

def parse_lyrics(text):
  letters = string.ascii_lowercase + ' ' + '\''
  new = text.lower()
  new = new.replace('\n', ' ')
  new = "".join([i for i in new if i in letters])
  # put a space at the start and at the end
  new = ' ' + new + ' '
  return new

text = parse_lyrics(text)
print(text)

 you would not believe your eyes if ten million fireflies lit up the world as i fell asleep 'cause they fill the open air and leave teardrops everywhere you'd think me rude but i would just stand and stare i'd like to make myself believe that planet earth turns slowly it's hard to say that i'd rather stay awake when i'm asleep 'cause everything is never as it seems 'cause i'd get a thousand hugs from ten thousand lightning bugs as they tried to teach me how to dance a foxtrot above my head a sockhop beneath my bed a disco ball is just hanging by a thread thread thread i'd like to make myself believe that planet earth turns slowly it's hard to say that i'd rather stay awake when i'm asleep 'cause everything is never as it seems when i fall asleep leave my door open just a crack please take me away from here 'cause i feel like such an insomniac please take me away from here why do i tire of counting sheep please take me away from here when i'm far too tired to fall asleep to ten million 

In [ ]:
letters = string.ascii_lowercase + ' ' + '\''
"".join([i for i in text if i in letters])

In [119]:
correct_text = minimal_token(text, decoder.labels)[1]

#speech_file = '/content/fireflies_voc_1min.wav'
speech_file = '/content/vocals.wav'
test_files = [speech_file]
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]))

graphemes = decoder.labels
res = tf_model.signatures["serving_default"](tf.constant(input.numpy()))['output_0']
text_from_speech = np.array(res)[0]

In [ ]:
text_from_speech.shape

In [ ]:
#forced_alignment_recursive(correct_text, text_from_speech, len(correct_text) - 1, time_values - 1)
#forced_alignment_recursive(correct_text, text_from_speech, 1, 1)
subproblems

In [120]:
result = forced_alignment(correct_text, text_from_speech)
result = [decoder.labels[correct_text[i]] for i in result]
print(result)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',

In [95]:
print(len(correct_text))
print(text_from_speech.shape)
test = np.array(subproblems)
print(test.shape)


825
(2855, 999)
(0,)


In [94]:
subproblems = []

In [114]:
decoder.labels.index(' ')

998

In [88]:
test = np.array(subproblems)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [89]:
test.shape

(3232,)

In [ ]:
test[10][10]

In [ ]:
text_from_speech

In [84]:
text_from_speech.shape

(2855, 999)

In [86]:
correct_text

[37,
 998,
 184,
 998,
 90,
 998,
 933,
 998,
 215,
 998,
 681,
 998,
 95,
 998,
 429,
 998,
 892,
 52,
 7,
 998,
 297,
 489,
 52,
 21,
 998,
 52,
 970,
 998,
 130,
 998,
 2,
 998,
 342,
 998,
 16,
 998,
 973,
 998,
 83,
 93,
 998,
 16,
 24,
 341,
 998,
 991,
 115,
 175,
 998,
 94,
 998,
 962,
 93,
 998,
 2,
 998,
 800,
 998,
 592,
 998,
 13,
 998,
 24,
 411,
 998,
 87,
 151,
 42,
 317,
 998,
 294,
 279,
 998,
 37,
 991,
 979,
 998,
 258,
 998,
 43,
 998,
 977,
 286,
 969,
 998,
 84,
 998,
 973,
 998,
 184,
 998,
 223,
 998,
 434,
 998,
 13,
 998,
 25,
 62,
 998,
 973,
 991,
 979,
 998,
 170,
 998,
 12,
 998,
 350,
 998,
 113,
 263,
 998,
 933,
 998,
 32,
 998,
 748,
 179,
 998,
 470,
 1,
 998,
 289,
 975,
 998,
 975,
 206,
 31,
 998,
 17,
 991,
 975,
 998,
 702,
 998,
 12,
 998,
 303,
 998,
 32,
 998,
 973,
 991,
 979,
 998,
 336,
 89,
 998,
 25,
 54,
 998,
 927,
 367,
 969,
 998,
 173,
 998,
 973,
 991,
 982,
 998,
 16,
 24,
 341,
 998,
 991,
 115,
 175,
 998,
 294,
 131,
 998,
 10,


# Make the subtitles

In [121]:
!pip install srt

  Created wheel for srt: filename=srt-3.4.1-cp36-none-any.whl size=20571 sha256=313b69cb765c3b45ff5324e5fc82d940cb49c860fb1012c6e743c75d62a7c4b3
  Stored in directory: /root/.cache/pip/wheels/85/9e/29/17bfa5b6d3ce3c026b6bdf49557a269e0ef869492a1bc700b2
Successfully built srt


In [123]:
subtitles = result

original_text = text
original_text = [i for i in original_text.split(" ") if i is not '']


In [147]:
current_word = 0
current_time = 0

time_stamps = []
in_a_word = False
for time, symbol in enumerate(subtitles):
  if symbol is not ' ' and not in_a_word:
    time_stamps.append((time, original_text[current_word]))
    in_a_word = True
    current_word += 1
  if symbol is ' ' and in_a_word:
    in_a_word = False

time_stamps.append((len(subtitles) - 1, 'FINSIH'))


In [148]:
time_stamps

[(275, 'you'),
 (277, 'would'),
 (280, 'not'),
 (285, 'believe'),
 (289, 'your'),
 (297, 'eyes'),
 (305, 'if'),
 (309, 'ten'),
 (313, 'million'),
 (327, 'fireflies'),
 (340, 'lit'),
 (345, 'up'),
 (348, 'the'),
 (350, 'world'),
 (355, 'as'),
 (357, 'i'),
 (360, 'fell'),
 (366, 'asleep'),
 (404, "'cause"),
 (409, 'they'),
 (413, 'fill'),
 (417, 'the'),
 (421, 'open'),
 (428, 'air'),
 (437, 'and'),
 (439, 'leave'),
 (445, 'teardrops'),
 (454, 'everywhere'),
 (462, "you'd"),
 (467, 'think'),
 (471, 'me'),
 (476, 'rude'),
 (480, 'but'),
 (483, 'i'),
 (487, 'would'),
 (491, 'just'),
 (494, 'stand'),
 (498, 'and'),
 (510, 'stare'),
 (540, "i'd"),
 (545, 'like'),
 (547, 'to'),
 (550, 'make'),
 (559, 'myself'),
 (566, 'believe'),
 (590, 'that'),
 (595, 'planet'),
 (598, 'earth'),
 (601, 'turns'),
 (635, 'slowly'),
 (666, "it's"),
 (670, 'hard'),
 (673, 'to'),
 (675, 'say'),
 (679, 'that'),
 (682, "i'd"),
 (686, 'rather'),
 (689, 'stay'),
 (700, 'awake'),
 (707, 'when'),
 (709, "i'm"),
 (718, '

In [152]:
import srt
from srt import Subtitle
import datetime


subs = []

for i, _ in enumerate(time_stamps):
  if i == len(time_stamps) - 1:
    break
  time_start = time_stamps[i][0] / 12.5
  time_end = time_stamps[i+1][0] / 12.5
  subs.append(Subtitle(index=i+1, start=datetime.timedelta(0, time_start, 0), end=datetime.timedelta(0, time_end, 0), content=time_stamps[i][1], proprietary=''))

print(srt.compose(subs))


1
00:00:22,000 --> 00:00:22,160
you

2
00:00:22,160 --> 00:00:22,400
would

3
00:00:22,400 --> 00:00:22,800
not

4
00:00:22,800 --> 00:00:23,120
believe

5
00:00:23,120 --> 00:00:23,760
your

6
00:00:23,760 --> 00:00:24,400
eyes

7
00:00:24,400 --> 00:00:24,720
if

8
00:00:24,720 --> 00:00:25,040
ten

9
00:00:25,040 --> 00:00:26,160
million

10
00:00:26,160 --> 00:00:27,200
fireflies

11
00:00:27,200 --> 00:00:27,600
lit

12
00:00:27,600 --> 00:00:27,840
up

13
00:00:27,840 --> 00:00:28,000
the

14
00:00:28,000 --> 00:00:28,400
world

15
00:00:28,400 --> 00:00:28,560
as

16
00:00:28,560 --> 00:00:28,800
i

17
00:00:28,800 --> 00:00:29,280
fell

18
00:00:29,280 --> 00:00:32,320
asleep

19
00:00:32,320 --> 00:00:32,720
'cause

20
00:00:32,720 --> 00:00:33,040
they

21
00:00:33,040 --> 00:00:33,360
fill

22
00:00:33,360 --> 00:00:33,680
the

23
00:00:33,680 --> 00:00:34,240
open

24
00:00:34,240 --> 00:00:34,960
air

25
00:00:34,960 --> 00:00:35,120
and

26
00:00:35,120 --> 00:00:35,600
l

In [135]:
time_stamps

[(275, 'you'),
 (277, 'would'),
 (280, 'not'),
 (285, 'believe'),
 (289, 'your'),
 (297, 'eyes'),
 (305, 'if'),
 (309, 'ten'),
 (313, 'million'),
 (327, 'fireflies'),
 (340, 'lit'),
 (345, 'up'),
 (348, 'the'),
 (350, 'world'),
 (355, 'as'),
 (357, 'i'),
 (360, 'fell'),
 (366, 'asleep'),
 (404, "'cause"),
 (409, 'they'),
 (413, 'fill'),
 (417, 'the'),
 (421, 'open'),
 (428, 'air'),
 (437, 'and'),
 (439, 'leave'),
 (445, 'teardrops'),
 (454, 'everywhere'),
 (462, "you'd"),
 (467, 'think'),
 (471, 'me'),
 (476, 'rude'),
 (480, 'but'),
 (483, 'i'),
 (487, 'would'),
 (491, 'just'),
 (494, 'stand'),
 (498, 'and'),
 (510, 'stare'),
 (540, "i'd"),
 (545, 'like'),
 (547, 'to'),
 (550, 'make'),
 (559, 'myself'),
 (566, 'believe'),
 (590, 'that'),
 (595, 'planet'),
 (598, 'earth'),
 (601, 'turns'),
 (635, 'slowly'),
 (666, "it's"),
 (670, 'hard'),
 (673, 'to'),
 (675, 'say'),
 (679, 'that'),
 (682, "i'd"),
 (686, 'rather'),
 (689, 'stay'),
 (700, 'awake'),
 (707, 'when'),
 (709, "i'm"),
 (718, '

In [136]:
subtitles

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' '